In [0]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()

drive=GoogleDrive(gauth)

In [0]:
train_clean = drive.CreateFile({'id': '1_y2mSjhcw0uaI_vTgrLdd0UScWIEsjIb'})
train_clean.GetContentFile('train_clean.csv')
test_clean = drive.CreateFile({'id': '1U0g_7u7g61UTywF0elODU06jyJ_wLfjF'})
test_clean.GetContentFile('test_clean.csv')

In [0]:
train = drive.CreateFile({'id': '1UAMpsYjI2161mmcs8MVsuEF-1okd43jT'})
train.GetContentFile('train.csv')
test = drive.CreateFile({'id': '17uPPXQbgzT772MCpsoddfEIYS95oGkD0'})
test.GetContentFile('test.csv')
sub = drive.CreateFile({'id': '1_48gjftaBNWfVd4DiMG4a7qQ-3Bv7P_N'})
sub.GetContentFile('sample_submission.csv')



In [0]:

############################imports##########################################


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tqdm
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import confusion_matrix, f1_score, plot_confusion_matrix
from keras.models import Model
import keras.layers as L
from keras.utils import to_categorical, plot_model
from scipy import signal

from sklearn.experimental import enable_hist_gradient_boosting  # noqa

from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,HistGradientBoostingClassifier,GradientBoostingClassifier
from sklearn.svm import SVC


import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose

import json
from google.colab import files


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
train_clean=pd.read_csv('train_clean.csv')
test_clean=pd.read_csv('test_clean.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
sub=pd.read_csv('sample_submission.csv')

In [0]:
batch_separated=[];batch_separated_clean_train=[];batch_separated_clean_test=[]
#train_clean['signal']=normalize(train_clean.signal)
#test_clean['signal']=normalize(test_clean.signal)
single_batch_size=500000
num_of_batch_train=int(train.shape[0]/single_batch_size)
num_of_batch_test=int(test.shape[0]/single_batch_size)

train_clean['group']=0;test_clean['group']=0
for i in range(num_of_batch_train):
  train_clean['group'].iloc[i*single_batch_size:(i+1)*single_batch_size]=i
  temp_df=train.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated.append(temp_df)
  temp_df=train_clean.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated_clean_train.append(temp_df)
for i in range(num_of_batch_test):
  test_clean['group'].iloc[i*single_batch_size:(i+1)*single_batch_size]=i
  temp_df=test_clean.iloc[single_batch_size*i:single_batch_size*(i+1)]
  batch_separated_clean_test.append(temp_df)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

#train=reduce_mem_usage(train)

# Feature Engineering

In [0]:
######## Feature generation#

def feature_engineering(s,windows_roll,windows_ewm,windows_lag):
  ewm = pd.DataFrame();high_pass = pd.DataFrame();low_pass = pd.DataFrame();grads = pd.DataFrame();merged_features=pd.DataFrame()
  roll_stats = pd.DataFrame();lead_lag=pd.DataFrame();column_names=[]
  n_filts=10
  wns = np.logspace(-2, -0.3, n_filts)
  n_grads=4
  for window in windows_roll:
        roll_stats['roll_mean_' + str(window)] = s.rolling(window=window, min_periods=1).mean()
        roll_stats['roll_std_' + str(window)] = s.rolling(window=window, min_periods=1).std()
        roll_stats['roll_min_' + str(window)] = s.rolling(window=window, min_periods=1).min()
        roll_stats['roll_max_' + str(window)] = s.rolling(window=window, min_periods=1).max()
        roll_stats['roll_range_' + str(window)] = roll_stats['roll_max_' + str(window)] - roll_stats['roll_min_' + str(window)]
        roll_stats['roll_q10_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.10)
        roll_stats['roll_q25_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.25)
        roll_stats['roll_q50_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.50)
        roll_stats['roll_q75_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.75)
        roll_stats['roll_q90_' + str(window)] = s.rolling(window=window, min_periods=1).quantile(0.90)
       # roll_stats['sig_mag' + str(window)] = s.rolling(window=window,min_periods=1).apply(lambda x: sum(abs(x))/single_batch_size)

  roll_stats = roll_stats.fillna(value=0)
  #print(roll_stats.shape)
  roll_stats=roll_stats.reset_index(drop=True)
  ################################################################################################################
  for window in windows_lag:
    lead_lag['lag_'+str(window)]=s.shift(window)
    lead_lag['lead_'+str(window)]=s.shift(-1*window)
  lead_lag=lead_lag.fillna(0)
  lead_lag=lead_lag.reset_index(drop=True)

  ################################################################################################################

  g=s.values
  for i in range(n_grads):
    g = np.gradient(g)
    grads['grad_' + str(i+1)] = g
  grads=grads.fillna(0)
  grads=grads.reset_index(drop=True)
  ################################################################################################################
 
  for wn in wns:
    b, a = signal.butter(1, Wn=wn, btype='low')
    low_pass['lowpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, s.values)
    low_pass['lowpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, s.values)
  low_pass=low_pass.fillna(0)
  low_pass=low_pass.reset_index(drop=True)
  ################################################################################################################
  
  for wn in wns:
    b, a = signal.butter(1, Wn=wn, btype='high')
    high_pass['hihgpass_lf_' + str('%.4f' %wn)] = signal.lfilter(b, a, s.values)
    high_pass['hihgpass_ff_' + str('%.4f' %wn)] = signal.filtfilt(b, a, s.values)  
  high_pass=high_pass.fillna(0)
  high_pass=high_pass.reset_index(drop=True)
  ################################################################################################################

  for window in windows_ewm:
    ewm['ewm_mean_' + str(window)] = s.ewm(span=window, min_periods=1).mean()
    ewm['ewm_std_' + str(window)] = s.ewm(span=window, min_periods=1).std()
  ewm=ewm.fillna(0)
  ewm=ewm.reset_index(drop=True)
  ################################################################################################################

  merged_features=pd.concat([roll_stats,lead_lag,grads,ewm,low_pass,high_pass],axis=1,ignore_index=True) 
  #merged_features=merged_features.append([roll_stats,lead_lag,grads,ewm,low_pass,high_pass],)
  #print(grads.shape,low_pass.shape,high_pass.shape,ewm.shape,lead_lag.shape,roll_stats.shape)
  #print("aaa")
  #merged_features=pd.concat([lead_lag,grads],axis=1,ignore_index=True) 

  column_names=list(roll_stats.columns)+list(lead_lag.columns)+list(grads.columns)+list(ewm.columns)+list(low_pass.columns)+list(high_pass.columns)

  #column_names=list(lead_lag.columns)+list(grads.columns)+list(ewm.columns)+list(low_pass.columns)+list(high_pass.columns)
  #print(merged_features.shape)
  return merged_features,column_names


In [0]:
windows_roll=[10,50,100,500,1000];windows_ewm=[10,50];windows_lag=[1,2,3];new_features=pd.DataFrame()
temp_df=pd.DataFrame()

for i in tqdm.tqdm(range(num_of_batch_train)):  
  temp_df,columns=feature_engineering(batch_separated_clean_train[i].signal,windows_roll,windows_ewm,windows_lag)
  #new_features=pd.concat([new_features,temp_df],axis=0,ignore_index=True)
  new_features=new_features.append(temp_df)
  
col_change=dict(zip(temp_df.columns,columns))
new_features=reduce_mem_usage(new_features)
new_features=new_features.reset_index(drop=True)
train=pd.concat([train_clean,new_features],axis=1)
train.rename(columns=col_change,inplace=True)
train['sig2']=train['signal']**2


100%|██████████| 10/10 [01:47<00:00, 10.80s/it]


Mem. usage decreased to 1029.97 Mb (74.3% reduction)


In [0]:
new_features=pd.DataFrame();temp_df=pd.DataFrame()

for i in tqdm.tqdm(range(num_of_batch_test)):  
  temp_df,columns=feature_engineering(batch_separated_clean_test[i].signal,windows_roll,windows_ewm,windows_lag)
  #new_features=pd.concat([new_features,temp_df],axis=0,ignore_index=True)
  new_features=new_features.append(temp_df)
  
col_change=dict(zip(temp_df.columns,columns))
new_features=reduce_mem_usage(new_features)
new_features=new_features.reset_index(drop=True)
test=pd.concat([test_clean,new_features],axis=1)
test.rename(columns=col_change,inplace=True)
test['sig2']=test['signal']**2

100%|██████████| 4/4 [00:38<00:00,  9.64s/it]


Mem. usage decreased to 411.99 Mb (74.3% reduction)


# Feature Selection

In [0]:

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [0]:
y=train.open_channels

X=train.drop(['time','open_channels','group'],axis=1)
FEATURES=X.columns
feature_importances_df= pd.DataFrame(index=FEATURES)
train_X,val_X,train_y,val_y =train_test_split(X,y,test_size=0.2)


In [0]:
names=['rf','extra']
models=[RandomForestClassifier(
            n_estimators=10,
            max_depth=19,
            #max_features=10,
            random_state=42,
            n_jobs=10,
            verbose=2),
        ExtraTreesClassifier(
            n_estimators=10,
            max_depth=19,
            #max_features=10,
            random_state=42,
            n_jobs=10,
            verbose=2)]
        
        #HistGradientBoostingClassifier(loss='categorical_crossentropy')]
classifiers_sel=dict(zip(names,models))
selected_features=dict(zip(names,[]))
cc=np.zeros((len(names),test.shape[0],train.open_channels.nunique()))
preds_=dict(zip(names,cc))
train_X,val_X,train_y,val_y =train_test_split(X,y,test_size=0.2)


In [0]:
for i in range(len(names)):
  classifiers_sel[names[i]].fit(train_X, train_y)
  f1=f1_score(val_y,classifiers_sel[names[i]].predict(val_X),average='macro')
  print("F1-Score for classifier "+names[i]," : ",f1)
  feature_importances_df['importance'] = classifiers_sel[names[i]].feature_importances_
  feature_importances_df=feature_importances_df.sort_values(by='importance',ascending=False)
  selected_features[names[i]]=feature_importances_df.index[:30]



[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 10building tree 2 of 10
building tree 3 of 10

building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  4.5min remaining: 10.5min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  5.3min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    1.4s remaining:    3.3s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    1.6s finished


F1-Score for classifier rf  :  0.9366018873248794


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 10building tree 2 of 10

building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.2min remaining:  2.8min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.4min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    1.6s remaining:    3.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    1.7s finished


F1-Score for classifier extra  :  0.9352787486172226


# Modeling

In [0]:
models=[RandomForestClassifier(
            n_estimators=100,
            max_depth=19,
            #max_features=10,
            random_state=42,
            n_jobs=10,
            verbose=2),
        ExtraTreesClassifier(
            n_estimators=100,
            max_depth=19,
            #max_features=10,
            random_state=42,
            n_jobs=10,
            verbose=2)]
classifiers_sel=dict(zip(names,models))


In [26]:
folds=5
kf = KFold(n_splits=folds, shuffle=True, random_state=667)
fold=1
models = []; preds_net=[]
for i in range(len(names)):
    preds_ = np.zeros((len(test), 11))
    print(f'====== Fold {fold:0.0f} of {folds} ======')
    for train_index, val_index in kf.split(X, y):
    
      train_X = X[selected_features[names[i]]].iloc[train_index]
      val_X = X[selected_features[names[i]]].iloc[val_index]
      train_y = y.iloc[train_index]
      val_y = y.iloc[val_index]
    
      classifiers_sel[names[i]].fit(train_X, train_y)
      f1=f1_score(val_y,classifiers_sel[names[i]].predict(val_X),average='macro')
      print(f"F1-Score for classifier of fold {fold} "+names[i]," : ",f1)
      preds_proba = classifiers_sel[names[i]].predict_proba(test[selected_features[names[i]]])
      preds_proba = preds_proba.reshape(-1, preds_proba.shape[-1])  
      print(type(preds_proba),preds_proba.shape)         
      preds_+=preds_proba / folds
    preds_net.append(preds_)  
    fold+=1


====== Fold 1 of 2 ======


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.


building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:  1.1min remaining:  2.6min
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:  1.2min finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.0s remaining:    4.7s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    2.4s finished


F1-Score for classifier of fold 1 rf  :  0.9357309882861263


...

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:   14.9s remaining:   34.7s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   15.7s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    2.2s remaining:    5.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    2.5s finished


F1-Score for classifier of fold 2 extra  :  0.9272122713725359


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed:    1.3s remaining:    3.1s


<class 'numpy.ndarray'> (2000000, 11)


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    1.7s finished


In [0]:
preds_net[0].shape

(2000000, 11)

In [0]:
sub['open_channels'] = np.argmax(preds_net[0], axis = 1).astype(int)
sub.to_csv('submission_rrrrrrrrrrrf.csv', index=False, float_format='%.4f')
files.download('submission_rrrrrrrrrrrf.csv')

In [0]:
sub['open_channels'] = np.argmax(preds_net[1], axis = 1).astype(int)
sub.to_csv('submission_extra_trees.csv', index=False, float_format='%.4f')
files.download('submission_extra_trees.csv')